In [ ]:
# library imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")

from pydap.net import create_session
from pydap.client import open_url
import pydap
import gsw_xarray as gsw_xr
import cf_xarray

# local imports
from helperLibrary import subset, crop, profileLocationPlot


In [ ]:
minimalProfileData = xr.open_dataset(
    "../data/ARGO_VortexProfiles/subsetProfiles/EasternTropicalPacific_densityMapped_minimal.nc"
)
minimalProfileData.info

In [ ]:
minimalProfileData['lon'][5738].values

In [ ]:
climatological

In [ ]:
profileMLD = xr.open_dataset(
    "../data/ARGO_MLD/Argo_mixedlayers_all_04142022.nc"
)

In [ ]:
climatologicalMLD = xr.open_dataset(
    "../data/ARGO_MLD/Argo_mixedlayers_monthlyclim_04142022.nc"
)

In [ ]:
climatologicalMLD['mld_da_max']

In [ ]:
climatologicalMLD['lon'][0].values

In [ ]:
climatologicalMLD['lon'].sel(
    iLON=179 + int(minimalProfileData['lon'][5738].values),
).values

In [ ]:
minimalProfileData['time'][5738].values

In [ ]:
climatologicalMLD['mld_da_max'].sel(
    iLAT=90 + int(minimalProfileData['lat'][5738].values),
    iLON=179 + int(minimalProfileData['lon'][5738].values),
    iMONTH=minimalProfileData['time'][5738].dt.month - 1,
).values

In [ ]:
minimalClimatology = xr.open_dataset(
    "../data/ARGO_RG2019_Climatology/subsetClimatology/RG_ArgoClim_ETP_densityMapped_minimal.nc"
)
minimalClimatology.info

In [ ]:
# Interpolate climatology to profile locations
profileLocationClimatology = minimalClimatology[['SA', 'sigma0']].interp(
    LATITUDE=('casts', minimalProfileData['lat'].data),
    LONGITUDE=('casts', (minimalProfileData['lon'].data + 360) % 360),
    kwargs={'fill_value': None}
)

In [ ]:
profileLocationClimatology['casts'] = ("casts", minimalProfileData['casts'].data)

In [ ]:
missingInterpolationCasts = profileLocationClimatology['casts'].data[
    np.isnan(profileLocationClimatology['SA'].data)[0,:,0]
]

In [ ]:
minimalProfileData

In [ ]:
castSA.data[:10]
castSigma0.data[:10]

In [ ]:
salinityAnomaly = minimalProfileData.copy()
salinityAnomaly['SA_anomaly'] = xr.DataArray(
    np.full_like(minimalProfileData['SA'].data, np.nan),
    dims=minimalProfileData['SA'].dims,
    coords=minimalProfileData['SA'].coords
)
salinityAnomaly['SA_climatology'] = xr.DataArray(
    np.full_like(minimalProfileData['SA'].data, np.nan),
    dims=minimalProfileData['SA'].dims,
    coords=minimalProfileData['SA'].coords
)

In [ ]:
salinityAnomaly.drop_vars(['SA', 'z'])

In [ ]:
np.isin(cast, missingInterpolationCasts)

In [ ]:
profileLocationClimatology

In [ ]:
for cast in profileLocationClimatology['casts'][:]:
    print(cast.values)

In [ ]:
casts = profileLocationClimatology['casts'][8472:8473]

for cast in casts:
    castSigma0 = minimalProfileData['sigma0'].sel(casts=cast)
    castSA = minimalProfileData['SA'].sel(casts=cast)
    castYear = minimalProfileData['time.year'].sel(casts=cast).data
    castMonth = minimalProfileData['time.month'].sel(casts=cast).data
    castClimatology = profileLocationClimatology.sel(
            casts=cast,
            TIME=slice(
                np.datetime64(f'{castYear}-{castMonth:02d}') -
                np.timedelta64(5, 'M'),
                np.datetime64(f'{castYear}-{castMonth:02d}') +
                np.timedelta64(6, 'M')
            )
        )
    
    plt.figure(figsize=(10, 6))
    plt.plot(
        castSA.data, castSigma0.data,
        label='Profile Data', marker='o', linestyle='-'
    )

    meanInterpolatedBackground = \
        np.full_like(castSA.data, np.nan)

    for time in castClimatology['TIME'].data:
        # Get the background field for the current time
        backgroundField = castClimatology.sel(TIME=time)

        # Interpolate the background field to the density surfaces
        # and accumulate the mean

        if np.isnan(meanInterpolatedBackground).all():
            meanInterpolatedBackground = np.interp(
                castSigma0.data,
                backgroundField['sigma0'].data,
                backgroundField['SA'].data,
                left=np.nan,  # Handle extrapolation
                right=np.nan   # Handle extrapolation
            )
        else:
            # Use np.nanmean to handle NaNs in the mean calculation
            stack = np.stack(
                (meanInterpolatedBackground,
                 np.interp(
                     castSigma0.data,
                     backgroundField['sigma0'].data,
                     backgroundField['SA'].data,
                     left=np.nan,  # Handle extrapolation
                     right=np.nan   # Handle extrapolation
                 )),
                axis=1
            )
            np.nanmean(
                stack, axis=1, out=meanInterpolatedBackground
            )
    plt.plot(
        meanInterpolatedBackground, castSigma0.data, linestyle='-',
        label=f'Climatology'
    )
    plt.title(f'Cast {cast.data} - {castYear}-{castMonth:02d}')
    plt.xlabel('Absolute Salinity (g/kg)')
    plt.ylabel('Potential Density (kg/m³)')
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
# Create a numpy array to hold the mean interpolated background
interpolatedBackground = \
    np.full_like(castClimatology['SA'].data, np.nan)

interpolatedBackground[:,12]

In [ ]:
salinityAnomaly['SA_anomaly'].loc[dict(casts=cast)] = (
            castSA - meanInterpolatedBackground
        )
salinityAnomaly['SA_anomaly'].loc[dict(casts=cast)].data[:10]

In [ ]:
data = minimalProfileData['casts'][np.argwhere(interpolatedSalinity[-1,:,-10].isnull().values)[:,0]]
fig, ax = plt.subplots(figsize=(10, 6), subplot_kw={'projection': ccrs.PlateCarree()})
scatter = ax.scatter(
    data['lon'], data['lat'], cmap='viridis'
)
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.LAND, edgecolor='black')
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.set_title('Cast locations with missing interpolated climatology')
ax.gridlines(
    draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--'
)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.savefig('missing_salinity_casts.png', dpi=300, bbox_inches='tight')

In [ ]:
salinityAnomalies = xr.open_dataset(
    "../data/ARGO_VortexProfiles/subsetProfiles/EasternTropicalPacific_anomalies.nc"
)

In [ ]:
lat_range = (12.5, 14)
lon_range = (-106, -104.5)

sampleBinAnomalies = subset(
    salinityAnomalies,
    lat_range=lat_range,
    lon_range=lon_range,
    var_names=['lat', 'lon', 'time']
)

In [ ]:
plottingStuff = sampleBinAnomalies.where(sampleBinAnomalies['sigma0'] > 24)
plt.figure(figsize=(10, 6))
plt.scatter(
    np.abs(plottingStuff['SA_anomaly'].data), plottingStuff['sigma0'].data,
    linestyle='-', color='black', s=0.1
)
plt.xlabel(r'Absolute Salinity Anomaly $|S^\prime| = |S - \{S\}|$')
plt.ylabel(r'$\sigma_\theta$')
plt.title(f'Salinity Anomalies in Sample Bin - {lat_range[0]}°N to {lat_range[1]}°N, {lon_range[0]}°W to {lon_range[1]}°W')
# Reverse y-axis
plt.gca().invert_yaxis()
plt.legend()
plt.show()

In [ ]:
anomaly_avg = np.sqrt(np.nanmean(sampleBinAnomalies['SA_anomaly']**2, axis=0))
plt.figure(figsize=(10, 6))
plt.plot(
    anomaly_avg, np.nanmean(sampleBinAnomalies['sigma0'].data, axis=0),
    marker='o', linestyle='-'
)

In [ ]:
uvel_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/uvel'
vvel_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/vvel'
theta_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/theta'
salt_url = 'http://apdrc.soest.hawaii.edu:80/dods/public_data/ECCO/ECCO2/cube92_daily/salt'
ECCO2_uvel = xr.open_dataset(
    uvel_url,
    engine='pydap'
)
ECCO2_vvel = xr.open_dataset(
    vvel_url,
    engine='pydap'
)
ECCO2_theta = xr.open_dataset(
    theta_url,
    engine='pydap'
)
ECCO2_salt = xr.open_dataset(
    salt_url,
    engine='pydap'
)

In [ ]:
ECCO2_uvel = ECCO2_uvel.sel(
    lat=slice(5, 23.75),
    lon=slice(245, 267.5),
    time=slice(np.datetime64('2004-01-01'), np.datetime64('2022-12-31')),
    drop=True
)

In [ ]:
ECCO2_vvel = ECCO2_vvel.sel(
    lat=slice(5, 23.75),
    lon=slice(245, 267.5),
    time=slice(np.datetime64('2004-01-01'), np.datetime64('2022-12-31')),
    drop=True
)

In [ ]:
ECCO2velocities = xr.merge(
    [ECCO2_uvel, ECCO2_vvel]
)

In [ ]:
ECCO2velocities

In [ ]:
ECCO2velocities = ECCO2velocities.where(
    (ECCO2velocities['lat'] <= 18) |
    (ECCO2velocities['lon'] <= 240), drop=True
)